In [68]:
PARAMS = {
    # Operating frequency (MHz)
    'oper_freq_mhz': {
        'category': 'mandatory',
        'type': (int, float),
        'range': (100, 86000),
    },

    # Antenna diameter (m)
    'diameter_m': {
        'category': 'optional',
        'type': (int, float),
        'range': (0.001, 99.999),
    },

    # Maximum main-lobe antenna gain (dBi)
    'max_gain_dbi': {
        'category': 'optional',
        'type': (int, float),
        'range': (-29.9, 89.9),
    },

    # 3dB beamwidth (degrees)
    'beamwidth_deg': {
        'category': 'optional',
        'type': (int, float),
        'range': (0.001, 179.999),
    },
}


In [69]:
    def set_params(user_params):
        """Validate and set parameters according to PARAMS."""
        validated_params = {}  # Reset params before setting

        for param, rules in PARAMS.items():
            value = user_params.get(param)

            # Handle Mandatory
            if rules['category'] == 'mandatory' and value is None:
                raise ValueError(f"Missing required parameter '{param}'")

            # Handle Optional
            if rules['category'] == 'optional' and value is None:
                continue  # If not provided, it remains unset

            # Handle Conditional
            if rules['category'] == 'conditional' and 'depends_on' in rules:
                for dep_param, dep_check in rules['depends_on'].items():
                    dep_value = user_params.get(dep_param)

                    # If dependency is a lambda, evaluate the condition
                    if callable(dep_check):
                        if not dep_check(dep_value):
                            continue  # Dependency not met, so skip validation
                    else:
                        if dep_value != dep_check:
                            continue  # Dependency not met, so skip validation

                    # If we reach this point, the dependency is satisfied, so we need to validate the parameter
                    if value is None:
                        raise ValueError(
                            f"Missing required parameter '{param}' because '{dep_param}' is set to '{dep_value}'")

            # Type Check
            if 'type' in rules and value is not None and not isinstance(value, rules['type']):
                raise TypeError(f"'{param}' must be of type {rules['type']} but got {type(value)}")

            # Range Check
            if 'range' in rules and value is not None and not (rules['range'][0] <= value <= rules['range'][1]):
                raise ValueError(f"'{param}' must be in range {rules['range']} but got {value}")

            # Allowed Values Check
            if 'allowed' in rules and value is not None and value not in rules['allowed']:
                raise ValueError(f"'{param}' must be one of {rules['allowed']} but got '{value}'")

            # Store validated parameter
            validated_params[param] = value

        return validated_params

In [70]:
user_params = {
    'oper_freq_mhz': 23000, 
    #'max_gain_dbi': 45, 
    #'diameter_m': 5, 
    'beamwidth_deg': 14,
}

try:
    result = set_params(user_params)
    print("✅ Valid Test Passed. Result:", result)
except Exception as e:
    print(f"❌ Valid Test Failed. Error: {e}")


✅ Valid Test Passed. Result: {'oper_freq_mhz': 23000, 'beamwidth_deg': 14}
